[參考](https://github.com/histockhero/youtube_code/blob/main/Part2_%E8%82%A1%E5%B8%82%E8%B3%87%E6%96%99%E4%B8%8B%E8%BC%89%EF%BC%86%E6%8A%80%E8%A1%93%E5%88%86%E6%9E%90/2.1%E8%82%A1%E5%B8%82%E8%B3%87%E6%96%99%E4%B8%8B%E8%BC%89/%E4%B8%8B%E8%BC%89%E6%AD%B7%E5%B9%B4%E5%8F%B0%E8%82%A1%E5%B8%82%E5%A0%B4%E6%88%90%E4%BA%A4%E8%B3%87%E8%A8%8A.ipynb)

In [1]:
import requests
from io import StringIO
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
import time

In [2]:
def get_stock_close(date, stock_no):
    url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock_no}"
    data = requests.get(url).json()
    if "data" in data:
        df = pd.DataFrame.from_dict(data["data"])
        df.columns = data['fields']
        return df["收盤價"].values
    else:
        return False

In [3]:
def get_corporation(date):
    url = f'https://www.twse.com.tw/rwd/zh/fund/T86?response=json&date={date}&selectType=ALL'
    data = requests.get(url).json()
    df = pd.DataFrame.from_dict(data["data"])
    df.columns = data['fields']
    return df

In [5]:
df = pd.read_csv("stock_price_history.csv")
stock_list = pd.read_csv("stock_list.csv")

In [ ]:
df.columns.astype(int).to_numpy()

In [23]:
stock_list = np.setdiff1d(stock_list, df.columns.astype(int).to_numpy())

In [24]:
stock_list.shape

(83,)

In [25]:
for stock in stock_list:
    close_list = np.array([])
    for month in tqdm(month_list):
        close = get_stock_close(month, stock)
        time.sleep(0.15)
        if type(close) == bool and close == False:
            continue
        close_list = np.append(close_list, close)
    if len(close_list) < 3700:
        close_list = np.append(
            close_list, np.array([np.nan] * (3700 - len(close_list)))
        )
    df[f"{stock}"] = close_list

100%|██████████| 176/176 [00:33<00:00,  5.18it/s]
/tmp/ipykernel_540/3020429731.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{stock}"] = close_list
100%|██████████| 176/176 [00:34<00:00,  5.06it/s]
/tmp/ipykernel_540/3020429731.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{stock}"] = close_list
100%|██████████| 176/176 [00:36<00:00,  4.88it/s]
/tmp/ipykernel_540/3020429731.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

In [26]:
for stock_no in df.columns:
    df[stock_no] = (
        df[stock_no]
        .replace("--", np.nan, regex=True)
        .replace(",", "", regex=True)
        .astype(float)
    )

In [63]:
df[:3587].to_csv("stock_price_history.csv",index=False)

In [3]:
# 1個月
def get_index_volume(date):
    url = f"https://www.twse.com.tw/exchangeReport/FMTQIK?response=json&date={date}"
    data = requests.get(url).json()
    df = pd.DataFrame.from_dict(data["data"])
    df.columns = data['fields']
    return df
def get_index_K(date):
    url = f"https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=json&date={date}"
    data = requests.get(url).json()
    df = pd.DataFrame.from_dict(data["data"])
    df.columns = data['fields']
    return df

In [4]:
def date_helper(x):
    x = str(int(x[:3])+1911)+x[3:]
    return x.replace("/", "-")

In [10]:
month_list = pd.date_range('2010-01-01','2024-08-01', freq='MS').strftime("%Y%m%d").tolist()    
# K = pd.DataFrame(columns=get_index_K(month_list[0]).columns)
# volume = pd.DataFrame(columns=get_index_volume(month_list[0]).columns)

In [6]:
for month in tqdm(month_list):
    K = pd.concat([K,get_index_K(month)], ignore_index=True)
    time.sleep(0.3)

100%|██████████| 176/176 [01:12<00:00,  2.43it/s]


In [7]:
for month in tqdm(month_list):
    volume = pd.concat([volume,get_index_volume(month)], ignore_index=True)
    time.sleep(0.3)

100%|██████████| 176/176 [01:01<00:00,  2.85it/s]


In [8]:
volume["成交股數"] = volume["成交股數"].replace(',','', regex=True).astype(int)
volume["成交金額"] = volume["成交金額"].replace(',','', regex=True).astype(int)
volume["成交筆數"] = volume["成交筆數"].replace(',','', regex=True).astype(int)
volume["發行量加權股價指數"] = volume["發行量加權股價指數"].replace(',','', regex=True).astype(float)
volume["漲跌點數"] = volume["漲跌點數"].replace(',','', regex=True).astype(float)
volume["日期"] = volume["日期"].apply(date_helper)

In [9]:
volume.to_csv("大盤成交.csv",index=False)

In [10]:
K["開盤指數"] = K["開盤指數"].replace(',','', regex=True).astype(float)
K["最高指數"] = K["最高指數"].replace(',','', regex=True).astype(float)
K["最低指數"] = K["最低指數"].replace(',','', regex=True).astype(float)
K["收盤指數"] = K["收盤指數"].replace(',','', regex=True).astype(float)
K["日期"] = K["日期"].apply(date_helper)

In [11]:
K.to_csv("大盤K棒.csv",index=False)

In [12]:
K = pd.read_csv("大盤K棒.csv")
volume = pd.read_csv("大盤成交.csv")
corporation = pd.read_csv("整體市場三大法人買賣表.csv")
market = pd.read_csv("整體市場融資融劵表.csv")

In [13]:
index = pd.DataFrame()
index["date"] = K["日期"]
index["open"] = K["開盤指數"]
index["high"] = K["最高指數"]
index["low"] = K["最低指數"]
index["close"] = K["收盤指數"]
index["volume_stockNum"] = volume["成交股數"]
index["volume_price"] = volume["成交金額"]
index["volume_num"] = volume["成交筆數"]
index.head()

,date,open,high,low,close,volume_stockNum,volume_price,volume_num
0,2010-01-04,8222.42,8240.45,8143.45,8207.85,6465473557,171654446230,1500109
1,2010-01-05,8277.71,8283.64,8162.54,8211.40,6795431855,197446584919,1613717
2,2010-01-06,8237.10,8327.87,8216.92,8327.62,6462699499,188531863020,1514442
3,2010-01-07,8344.56,8369.55,8233.59,8237.42,7183703184,203297890690,1690439
4,2010-01-08,8266.87,8290.66,8178.93,8280.90,5837432488,161862787984,1382306


In [14]:
def market_handler():
    market_type = ["MarginPurchase","ShortSale","MarginPurchaseMoney"]
    market_data = pd.DataFrame()
    market_data["date"] = market[market["種類"] == market_type[0]]["日期"].values
    for i in market_type:
        market_data[f"{i}_buy"] = market[market["種類"] == i]["買進"].values
        market_data[f"{i}_sell"] = market[market["種類"] == i]["賣出"].values
        market_data[f"{i}_val"] = market[market["種類"] == i]["現金/券償還"].values
    return market_data

In [15]:
def corporation_handler():
    corporation_type = ["Foreign_Investor","Investment_Trust"]
    corporation_data = pd.DataFrame()
    corporation_data["date"] = corporation[corporation["種類"] == corporation_type[0]]["日期"].values
    for i in corporation_type:
        corporation_data[f"{i}_buy"] = corporation[corporation["種類"] == i]["買進"].values
        corporation_data[f"{i}_sell"] = corporation[corporation["種類"] == i]["賣出"].values
    return corporation_data

In [16]:
corporation_data = corporation_handler()
market_data = market_handler()
data = pd.concat(
    [index, corporation_data.drop(columns="date"), market_data.drop(columns="date")],
    axis=1,
)

In [17]:
data.to_csv("data.csv",index=False)